## Load dataframes 

In [1]:
from pyspark.sql import SparkSession

# instantiate spark driver node
spark = SparkSession \
    .builder \
    .appName("Spark SQL Query Dataframes") \
    .getOrCreate()

# load csv file
json_df2_path = './data/utilization.json'
df = spark.read.format('json').load(json_df2_path)

# print out dataframe
df.show()

24/11/19 17:40:05 WARN Utils: Your hostname, Doomzies-2.local resolves to a loopback address: 127.0.0.1; using 192.168.68.101 instead (on interface en0)
24/11/19 17:40:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 17:40:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/19 17:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|            0.8|04/08/2019 23:37:13|       0.15|      133|           68|
|           0.84|04/08/2019 23:42:13|       0.14|      133|           66|
|           0.69|04/08/2019 23:47:13|        0.4|      133|           72|
|           0.75|04/08/2019 23:52:13|       0.18|      133|           77|
|           0.63|04/08/2019 23:57:13|       0.16|      133|           82|
|           0.72|04/09/2019 00:02:13|       0.06|      133|           78|
|            0.9|04/09/2019 00:07:13|       0.37|      133|           68|
|           0.59|04/09/2019 00:12:13|        0.4|      133|           94|
|           0.87|04/09/2019 00:17:13|       0.35|      133|           86|
|           0.83|04/09/2019 00:22:13|       0.34|      133|           92|
|           0.71|04/09/2019 00:27:13| 

24/11/19 17:40:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# determine count of rows in data frame
df.count()

500000

In [3]:
# prints the schema of table out
df.printSchema()

root
 |-- cpu_utilization: double (nullable = true)
 |-- event_datetime: string (nullable = true)
 |-- free_memory: double (nullable = true)
 |-- server_id: long (nullable = true)
 |-- session_count: long (nullable = true)



## Running SQL commands on table

In [7]:
# rename the table to 'utilization'
df.createOrReplaceTempView("utilization")

# apply SQL query on table utilization
df_sql = spark.sql("SELECT * FROM utilization LIMIT 10")
df_sql.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|            0.8|04/08/2019 23:37:13|       0.15|      133|           68|
|           0.84|04/08/2019 23:42:13|       0.14|      133|           66|
|           0.69|04/08/2019 23:47:13|        0.4|      133|           72|
|           0.75|04/08/2019 23:52:13|       0.18|      133|           77|
|           0.63|04/08/2019 23:57:13|       0.16|      133|           82|
|           0.72|04/09/2019 00:02:13|       0.06|      133|           78|
|            0.9|04/09/2019 00:07:13|       0.37|      133|           68|
|           0.59|04/09/2019 00:12:13|        0.4|      133|           94|
|           0.87|04/09/2019 00:17:13|       0.35|      133|           86|
|           0.83|04/09/2019 00:22:13|       0.34|      133|           92|
+---------------+-------------------+-

In [8]:
# selecting specific features
df_sql = spark.sql(
    "SELECT server_id as sid, session_count as sc FROM utilization")
df_sql.show()

+---+---+
|sid| sc|
+---+---+
|133| 68|
|133| 66|
|133| 72|
|133| 77|
|133| 82|
|133| 78|
|133| 68|
|133| 94|
|133| 86|
|133| 92|
|133| 94|
|133| 69|
|133| 88|
|133| 90|
|133| 96|
|133| 66|
|133| 77|
|133| 91|
|133| 84|
|133| 67|
+---+---+
only showing top 20 rows



In [11]:
# filtering for SQL using WHERE clause
df_sql = spark.sql("SELECT * \
                   FROM utilization \
                   WHERE session_count > 70 AND server_id = 120 \
                   ORDER BY session_count DESC")
df_sql.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|            0.6|03/06/2019 08:26:48|       0.39|      120|           80|
|           0.72|03/07/2019 18:21:48|       0.46|      120|           80|
|           0.65|03/06/2019 08:51:48|       0.31|      120|           80|
|           0.58|03/05/2019 19:16:48|       0.56|      120|           80|
|           0.44|03/06/2019 10:01:48|       0.41|      120|           80|
|            0.7|03/05/2019 08:21:48|       0.35|      120|           80|
|           0.65|03/06/2019 10:56:48|       0.45|      120|           80|
|           0.72|03/05/2019 23:26:48|       0.64|      120|           80|
|           0.61|03/06/2019 11:31:48|       0.25|      120|           80|
|           0.71|03/06/2019 23:56:48|       0.63|      120|           80|
|           0.66|03/07/2019 00:51:48| 

In [12]:
# aggregate functions

# determine count of entries where session_count > 70
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [14]:
# complex aggregate function
df_sql = spark.sql("SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      

In [16]:
# Joining two tables together

# create new table
csv_df_path = "./data/server_name.csv"
df_server = spark.read.csv(csv_df_path, header=True)

# rename table to server_name in SQL
df_server.createOrReplaceTempView("server_name")

# print table
df_server.show()

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
|      105| 105 Server|
|      106| 106 Server|
|      107| 107 Server|
|      108| 108 Server|
|      109| 109 Server|
|      110| 110 Server|
|      111| 111 Server|
|      112| 112 Server|
|      113| 113 Server|
|      114| 114 Server|
|      115| 115 Server|
|      116| 116 Server|
|      117| 117 Server|
|      118| 118 Server|
|      119| 119 Server|
+---------+-----------+
only showing top 20 rows



In [17]:
# attempt an SQL request with df_server
df_count = spark.sql(
    "SELECT DISTINCT server_id \
     FROM utilization \
     ORDER BY server_id")
df_count.show()

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
|      110|
|      111|
|      112|
|      113|
|      114|
|      115|
|      116|
|      117|
|      118|
|      119|
+---------+
only showing top 20 rows



In [18]:
# Join between utilization and server_name
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                     FROM utilization u \
                     INNER JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show()

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      133| 133 Server|           68|
|      133| 133 Server|           66|
|      133| 133 Server|           72|
|      133| 133 Server|           77|
|      133| 133 Server|           82|
|      133| 133 Server|           78|
|      133| 133 Server|           68|
|      133| 133 Server|           94|
|      133| 133 Server|           86|
|      133| 133 Server|           92|
|      133| 133 Server|           94|
|      133| 133 Server|           69|
|      133| 133 Server|           88|
|      133| 133 Server|           90|
|      133| 133 Server|           96|
|      133| 133 Server|           66|
|      133| 133 Server|           77|
|      133| 133 Server|           91|
|      133| 133 Server|           84|
|      133| 133 Server|           67|
+---------+-----------+-------------+
only showing top 20 rows



In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import SparkSession

# create spark driver node
spark = SparkSession.builder.getOrCreate()
spark_context = spark.sparkContext

# modify spark context
df_dup = spark_context.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80),
                                    Row(server_name='101 Server',
                                        cpu_utilization=80, session_count=90),
                                    Row(server_name='102 Server',
                                        cpu_utilization=85, session_count=80),
                                    Row(server_name='102 Server', cpu_utilization=85, session_count=80)]).toDF()

# print out df_dup
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [26]:
# drop exact duplicates
df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [25]:
# drop duplicates in server_name feature
df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [30]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# new dataframe
df = spark_context.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80),
                                Row(server_name='101 Server',
                                    cpu_utilization=80, session_count=90),
                                Row(server_name='102 Server',
                                    cpu_utilization=85, session_count=40),
                                Row(server_name='103 Server',
                                    cpu_utilization=70, session_count=80),
                                Row(server_name='104 Server', cpu_utilization=60, session_count=80)]).toDF()

# print out df
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           40|
| 103 Server|             70|           80|
| 104 Server|             60|           80|
+-----------+---------------+-------------+



In [32]:
# create new column in df 
df_na = df.withColumn('na_col', lit(None).cast(StringType()))
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           40|  NULL|
| 103 Server|             70|           80|  NULL|
| 104 Server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [35]:
# fill the na with 'A'
df_na.fillna('A').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [40]:
# union the df_na with the one that's filled
df2 = df_na.fillna('A').union(df_na)
df2.createOrReplaceTempView("na_table")
df2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           40|  NULL|
| 103 Server|             70|           80|  NULL|
| 104 Server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [39]:
# drop entries with NULL attributes
df2.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [41]:
# print out table with NULL in na_col
spark.sql("SELECT * FROM na_table WHERE na_col IS NULL").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           40|  NULL|
| 103 Server|             70|           80|  NULL|
| 104 Server|             60|           80|  NULL|
+-----------+---------------+-------------+------+

